In [ ]:
import numpy as np
import torch 
import gym 


In [ ]:
import matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2

In [ ]:
from preprocessing_wrappers import make_atari, wrap_deepmind, ImageToPyTorch

In [ ]:
env = gym.make('PongNoFrameskip-v4')
env.observation_space

In [ ]:
env_mapper = {'Pong':'PongNoFrameskip-v4',
               'Breakout': 'BreakoutNoFrameskip-v4',
               'Atlantis':'AtlantisNoFrameskip-v4'}


In [ ]:
obs = env.reset()

In [ ]:
np.array(obs).shape

In [ ]:
wrapped_env = ImageToPyTorch(wrap_deepmind(make_atari(env_mapper['Pong'])))

In [ ]:
wrapped_env.action_space

In [ ]:
wrapped_env.observation_space

In [ ]:
obs = wrapped_env.reset()

In [ ]:
wrapped_env.observation_space.shape

In [ ]:
np_obs = np.array(obs)

In [ ]:
np_obs.shape

In [ ]:
plt.imshow(np_obs[0,:,:])

In [ ]:
plt.imshow(np_obs[1,:,:])

In [ ]:
plt.imshow(np_obs[2,:,:])

In [ ]:
plt.imshow(np_obs[3,:,:,])

In [ ]:
obs, reward, done, info = wrapped_env.step(0)

In [ ]:
obs, reward, done, info = wrapped_env.step(1)

In [ ]:
plt.imshow(np_obs[:,:,0])

In [ ]:
obs.shape

In [ ]:
plt.imshow(np_obs[0,:,:])

In [ ]:
obs, reward, done, info = wrapped_env.step(3)

In [ ]:
np_obs = np.array(obs)

In [ ]:
np_obs.shape

In [ ]:
plt.imshow(np_obs[:,:,0])

In [ ]:
plt.imshow(np_obs[:,:,1])

In [ ]:
plt.imshow(np_obs[:,:,2])

In [ ]:
plt.imshow(np_obs[:,:,3])

In [ ]:
np.random.rand(1)[0]

In [ ]:
np.random.rand(1)

In [ ]:
x = np.array([1,2,3,4,5])
y = np.array([1000,989,979,400,800])

In [ ]:
plt.plot(x,y)
plt.savefig('tester.png')

In [ ]:
from train_utils import LinearSchedule

epsilon_schedule = LinearSchedule(1e6,1,0.1)

In [ ]:
timestep_inputs = list(range(1,int(1e6)))
epsilon_vals_out = []


In [ ]:
for t in timestep_inputs:
    eps_out = epsilon_schedule.get_epsilon(t)
    epsilon_vals_out.append(eps_out)

In [ ]:
plt.plot(epsilon_vals_out)

In [ ]:
capacity = 1000
state_dims = (4,84,84)

storage_size = (capacity,*state_dims)

In [ ]:
storage_size

In [ ]:
capacity = 1000
indices = list(range(capacity))

selected_indices = np.random.choice(indices)
selected_indices

